In [74]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import io
from google.colab import files
uploaded = files.upload()

Saving TrafficVolume_TrainingData.csv to TrafficVolume_TrainingData (1).csv


In [75]:
###--------- Import Datasets ---------###
dataset_train = pd.read_csv(io.BytesIO(uploaded['TrafficVolume_TrainingData.csv']))
# print(type(dataset_train))
training_set = dataset_train.iloc[:288, 2].values # Traffic volume column
                                                  # 2 because we want the TrafficVolume
                                                  # 288 because that gets all rows up until our last that has data

# make training_set_scaled a pandas dataframe object
training_set = pd.DataFrame(training_set, columns = ['TrafficVolume'])
# print(type(training_set))
# print(training_set)

###--------- Feature Scaling ---------###                                                  
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(-1, 1))
training_set['TrafficVolume'] = scaler.fit_transform(training_set['TrafficVolume'].values.reshape(-1,1))

In [76]:
def split_data(training_set, lookback):
    # data_raw = training_set_scaled.to_numpy() # convert to numpy array
    data_raw = training_set
    data = []
    
    # create all possible sequences of length seq_len
    for index in range(len(data_raw) - lookback): 
        data.append(data_raw[index: index + lookback])
    
    data = np.array(data);
    test_set_size = int(np.round(0.2*data.shape[0]));
    train_set_size = data.shape[0] - (test_set_size);
    
    x_train = data[:train_set_size,:-1,:]
    y_train = data[:train_set_size,-1,:]
    
    x_test = data[train_set_size:,:-1]
    y_test = data[train_set_size:,-1,:]
    
    return [x_train, y_train, x_test, y_test]
lookback = 24 # Lookback over the past 24 hours
x_train, y_train, x_test, y_test = split_data(training_set, lookback)

In [77]:
###--------- Setting Up PyTorch Model ---------###
import torch
import torch.nn as nn
x_train = torch.from_numpy(x_train).type(torch.Tensor)
x_test = torch.from_numpy(x_test).type(torch.Tensor)
y_train_lstm = torch.from_numpy(y_train).type(torch.Tensor)
y_test_lstm = torch.from_numpy(y_test).type(torch.Tensor)
y_train_gru = torch.from_numpy(y_train).type(torch.Tensor)
y_test_gru = torch.from_numpy(y_test).type(torch.Tensor)

In [78]:
# Define common values for both models regarding the layers
input_dim = 1
hidden_dim = 32
num_layers = 2
output_dim = 1
num_epochs = 100

In [79]:
# GRU model implementation
class GRU(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, output_dim):
        super(GRU, self).__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        
        self.gru = nn.GRU(input_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).requires_grad_()
        out, (hn) = self.gru(x, (h0.detach()))
        out = self.fc(out[:, -1, :]) 
        return out

In [80]:
# Creating the GRU model and setting the parameters
model = GRU(input_dim=input_dim, hidden_dim=hidden_dim, output_dim=output_dim, num_layers=num_layers)
criterion = torch.nn.MSELoss(reduction='mean')
optimiser = torch.optim.Adam(model.parameters(), lr=0.01)

In [81]:
# Train the model over 100 epochs
import time
hist = np.zeros(num_epochs)
start_time = time.time()
lstm = []
for t in range(num_epochs):
    y_train_pred = model(x_train)
    loss = criterion(y_train_pred, y_train_lstm)
    print("Epoch ", t, "MSE: ", loss.item())
    hist[t] = loss.item()
    optimiser.zero_grad()
    loss.backward()
    optimiser.step()
    
training_time = time.time()-start_time
print("Training time: {}".format(training_time))

Epoch  0 MSE:  0.35979920625686646
Epoch  1 MSE:  0.2933856248855591
Epoch  2 MSE:  0.26598596572875977
Epoch  3 MSE:  0.22020475566387177
Epoch  4 MSE:  0.18849614262580872
Epoch  5 MSE:  0.16006433963775635
Epoch  6 MSE:  0.1208593025803566
Epoch  7 MSE:  0.08914951980113983
Epoch  8 MSE:  0.06844939291477203
Epoch  9 MSE:  0.05211644992232323
Epoch  10 MSE:  0.049809034913778305
Epoch  11 MSE:  0.05812762305140495
Epoch  12 MSE:  0.05848823860287666
Epoch  13 MSE:  0.0526561513543129
Epoch  14 MSE:  0.04766559973359108
Epoch  15 MSE:  0.043153610080480576
Epoch  16 MSE:  0.038790032267570496
Epoch  17 MSE:  0.036781009286642075
Epoch  18 MSE:  0.03717736154794693
Epoch  19 MSE:  0.03796710819005966
Epoch  20 MSE:  0.037725239992141724
Epoch  21 MSE:  0.03695574402809143
Epoch  22 MSE:  0.03692493215203285
Epoch  23 MSE:  0.03761892020702362
Epoch  24 MSE:  0.03773605450987816
Epoch  25 MSE:  0.03677476570010185
Epoch  26 MSE:  0.03566046804189682
Epoch  27 MSE:  0.03519503027200699
